In [1]:
import sys
sys.path.append('..')
import pickle
import csv
import os
import pandas as pd
import yaml
import math
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from src.embedding import WordToVec as Embedding_Model
from src.DataLoader import LoadToken, GenerateLabels, LoadPickleData, SavedPickle
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Embedding
import torch.optim as optim
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from src.plstm_cell import PLSTM
from torch.utils.data import TensorDataset, DataLoader
import time

tokenizer_saved_path = 'embedding/'
repre_saved_path = 'Representation/'
data_path = 'data/ALL/VLC/'
test_size = 0.2
validation_size = 0.2
config = yaml.safe_load(open('config/config.yaml', 'r'))
random_seed = config['embedding_settings']['seed']
def verbose(msg):
    ''' Verbose function for print information to stdout'''
    print('[INFO]', msg)
    
def loadData(data_path):
    ''' Load data for training/validation'''
    verbose('Loading data from '+ os.getcwd() + os.sep + data_path + '....')
    total_list, total_list_id = LoadToken(data_path)
    verbose("The length of the loaded data list is : " + str(len(total_list)))
    return total_list, total_list_id

def Tokenization(data_list):
    tokenizer = Tokenizer(num_words=None, filters=',', lower=False, char_level=False, oov_token=None) 
    tokenizer.fit_on_texts(data_list)
    # Save the tokenizer.
    with open(tokenizer_saved_path + 'tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle)
    
def LoadToknizer(path_of_tokenizer):
    tokenizer = LoadPickleData(path_of_tokenizer)
    return tokenizer

def padding(sequences_to_pad):
    padded_seq = pad_sequences(sequences_to_pad, maxlen = config['model_settings']['model_para']['max_sequence_length'], padding ='post')
    return padded_seq

def patitionData(data_list_pad, data_list_id):
    
    test_size = config['training_settings']['dataset_config']['Test_set_ratio']
    validation_size = config['training_settings']['dataset_config']['Validation_set_ratio'] 
    data_list_label = GenerateLabels(data_list_id)
        
    if not config['training_settings']['using_separate_test_set']:
        # The value of the seed for testing should be the same to that was used during the training phase.  
        train_vali_set_x, test_set_x, train_vali_set_y, test_set_y, train_vali_set_id, test_set_id = train_test_split(data_list_pad, data_list_label, data_list_id, test_size=test_size, random_state=random_seed)
#         train_set_x, validation_set_x, train_set_y, validation_set_y, train_set_id, validation_set_id = train_test_split(train_vali_set_x, train_vali_set_y, train_vali_set_id, test_size=validation_size, random_state=random_seed)
        
        #tuple_with_test = train_set_x, train_set_y, train_set_id, validation_set_x, validation_set_y, validation_set_id, test_set_x, test_set_y, test_set_id
        tuple_with_test = train_vali_set_x, train_vali_set_y, test_set_x, test_set_y
        return tuple_with_test
    else:
        train_set_x, validation_set_x, train_set_y, validation_set_y, train_set_id, validation_set_id = train_test_split(train_vali_set_x, train_vali_set_y, train_vali_set_id, test_size=validation_size, random_state=random_seed)
        tuple_without_test = train_set_x, train_set_y, train_set_id, validation_set_x, validation_set_y, validation_set_id
        return tuple_without_test
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
total_list, total_list_id = loadData(data_path)
verbose("Perform tokenization ....")
Tokenization(total_list)
verbose("Tokenization completed!")
verbose("-------------------------------------------------------")   
verbose("Perform code embedding ....")
embedding_model = Embedding_Model(config)
'''total_sequnces: variable lengths of number of samples vectors'''
'''word_index: a map of word to vectors'''
total_sequences, word_index = embedding_model.LoadTokenizer(total_list)
embedding_model.TrainWordToVec(total_list)
embedding_matrix, embedding_dim = embedding_model.ApplyWordToVec(word_index)            
verbose ("Word2vec loaded! ")
            
verbose("Pad the sequence to unified length...")
total_list_pad = padding(total_sequences)
verbose("Patition the data ....")
data_tuple = patitionData(total_list_pad, total_list_id)  
        
train_set_x = data_tuple[0]
train_set_y = np.asarray(data_tuple[1]).flatten()

test_set_x = data_tuple[2]
test_set_y = np.asarray(data_tuple[3]).flatten()

train_set = train_set_x, train_set_y
test_set = test_set_x, test_set_y
y_test = test_set_y

train_set_x = torch.Tensor(train_set_x).long().to(device) # transform to torch tensor
train_set_y = torch.Tensor(train_set_y).to(device)
train_dataset = TensorDataset(train_set_x,train_set_y) # create your datset
train_loader = DataLoader(train_dataset, batch_size=128) # create your dataloader

test_set_x = torch.Tensor(test_set_x).long().to(device) # transform to torch tensor
test_set_y = torch.Tensor(test_set_y).to(device)
test_dataset = TensorDataset(test_set_x,test_set_y) # create your datset
test_loader = DataLoader(test_dataset, batch_size=1) # create your dataloader

verbose ("Data Shapes Are:")
verbose ("Data processing completed!")
verbose ("-------------------------------------------------------")
verbose ("There are " + str(len(train_set_x)) + " total samples in the training set. " + str(torch.count_nonzero(train_set_y)) + " vulnerable samples. ")
verbose ("There are " + str(len(test_set_x)) + " total samples in the test set. " + str(torch.count_nonzero(test_set_y)) + " vulnerable samples. ")

# if config['model_settings']['model_para']['handle_data_imbalance']:
#     class_weights = class_weight.compute_class_weight('balanced',
#                                             torch.unique(train_set_y),
#                                             train_set_y)
# else:
#     class_weights = None
        
# class_weight_dict = dict(zip([x for x in np.unique(train_set_y)], class_weights))
        
# verbose ("-------------------------------------------------------")

weights=torch.from_numpy(embedding_matrix)

# train_set_y = torch.FloatTensor(train_set_y).to(device)
#has shape of [1, 10000] and [1]

[INFO] Loading data from /home/swj/VD/data/ALL/VLC/....
[INFO] The length of the loaded data list is : 6159
[INFO] Perform tokenization ....
[INFO] Tokenization completed!
[INFO] -------------------------------------------------------
[INFO] Perform code embedding ....
----------------------------------------
Start training the Word2Vec model. Please wait.. 
Model training completed!
----------------------------------------
The trained word2vec model: 
Word2Vec(vocab=623, vector_size=100, alpha=0.025)
-------------------------------------------------------
Loading trained Word2vec model. 
The trained word2vec model: 
<_io.TextIOWrapper name='embedding/w2v_model.txt' mode='r' encoding='UTF-8'>
Found 624 word vectors.
[INFO] Word2vec loaded! 
[INFO] Pad the sequence to unified length...
[INFO] Patition the data ....
[INFO] Data Shapes Are:
[INFO] Data processing completed!
[INFO] -------------------------------------------------------
[INFO] There are 4927 total samples in the training s

In [2]:
OFF_SLOPE=1e-3

# function to extract grad
def set_grad(var):
    def hook(grad):
        var.grad = grad
    return hook

class GradMod(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """
    @staticmethod
    def forward(ctx, input, other):
        """
        In the forward pass we receive a Tensor containing the input and return a
        Tensor containing the output. You can cache arbitrary Tensors for use in the
        backward pass using the save_for_backward method.
        """
        result = torch.fmod(input, other)
        ctx.save_for_backward(input, other)        
        return result

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        x, y = ctx.saved_variables
        return grad_output * 1, grad_output * torch.neg(torch.floor_divide(x, y))
class PLSTMCell(nn.Module):
    def __init__(self, input_sz, hidden_sz):
        super().__init__()
        self.input_sz = input_sz
        self.hidden_size = hidden_sz
        self.Periods = nn.Parameter(torch.Tensor(hidden_sz, 1))
        self.Shifts = nn.Parameter(torch.Tensor(hidden_sz, 1))
        self.On_End = nn.Parameter(torch.Tensor(hidden_sz, 1))
        self.W = nn.Parameter(torch.Tensor(input_sz, hidden_sz * 4))
        self.U = nn.Parameter(torch.Tensor(hidden_sz, hidden_sz * 4))
        self.bias = nn.Parameter(torch.Tensor(hidden_sz * 4))
        self.init_weights()
                
    def init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)
        # Phased LSTM
        # -----------------------------------------------------
        nn.init.constant_(self.On_End, 0.05) # Set to be 5% "open"
        nn.init.uniform_(self.Shifts, 0, 100) # Have a wide spread of shifts
        # Uniformly distribute periods in log space between exp(1, 3)
        self.Periods.data.copy_(torch.exp((3 - 1) *
            torch.rand(self.Periods.shape) + 1))
        # -----------------------------------------------------
         
    def forward(self, t, x, ts,
                init_states=None):
        """Assumes x is of shape (batch, sequence, feature)"""
        bs, seq_sz, _ = x.size()
        if init_states is None:
            h_t, c_t = (torch.zeros(bs, self.hidden_size).to(x.device), 
                        torch.zeros(bs, self.hidden_size).to(x.device))
        else:
            h_t, c_t = init_states

        # PHASED LSTM
        # -----------------------------------------------------
        # Precalculate some useful vars
        shift_broadcast = self.Shifts.view(1, -1)
        period_broadcast = abs(self.Periods.view(1, -1))
        on_mid_broadcast = abs(self.On_End.view(1, -1)) * 0.5 * period_broadcast
        on_end_broadcast = abs(self.On_End.view(1, -1)) * period_broadcast                       
        
        def calc_time_gate(time_input_n):
            # Broadcast the time across all units
            t_broadcast = time_input_n.unsqueeze(-1)
            # Get the time within the period
            in_cycle_time = GradMod.apply(t_broadcast + shift_broadcast, period_broadcast)            

            # Find the phase
            is_up_phase = torch.le(in_cycle_time, on_mid_broadcast)
            is_down_phase = torch.gt(in_cycle_time, on_mid_broadcast)*torch.le(in_cycle_time, on_end_broadcast)


            # Set the mask
            sleep_wake_mask = torch.where(is_up_phase, in_cycle_time/on_mid_broadcast,
                                torch.where(is_down_phase,
                                    (on_end_broadcast-in_cycle_time)/on_mid_broadcast,
                                        OFF_SLOPE*(in_cycle_time/period_broadcast)))
            return sleep_wake_mask
        # -----------------------------------------------------

        HS = self.hidden_size
        old_c_t, old_h_t = c_t, h_t
        x_t = x[:, t, :]
        t_t = ts[:, t]
        
        # batch the computations into a single matrix multiplication
        gates = x_t @ self.W + h_t @ self.U + self.bias
        i_t, f_t, g_t, o_t = (
            torch.sigmoid(gates[:, :HS]), # input
            torch.sigmoid(gates[:, HS:HS*2]), # forget
            torch.tanh(gates[:, HS*2:HS*3]),
            torch.sigmoid(gates[:, HS*3:]), # output
        )
        c_t = f_t * c_t + i_t * g_t
        h_t = o_t * torch.tanh(c_t)
        # PHASED LSTM
        # -----------------------------------------------------
        # Get time gate openness
        sleep_wake_mask = calc_time_gate(t_t)
        # Sleep if off, otherwise stay a bit on
        c_t = sleep_wake_mask*c_t + (1. - sleep_wake_mask)*old_c_t
        h_t = sleep_wake_mask*h_t + (1. - sleep_wake_mask)*old_h_t
        # -----------------------------------------------------
        return h_t, c_t

class BiPLSTM(nn.Module):
    def __init__(self, input_sz, hidden_sz):
        super().__init__()
        self.input_sz = input_sz
        self.hidden_size = hidden_sz
        self.plstm_cell_forward = PLSTMCell(self.input_sz, self.hidden_size).to(device)
        self.plstm_cell_backward = PLSTMCell(self.input_sz, self.hidden_size).to(device)
    def forward(self, x, ts,
               init_states=None):
        bs, seq_sz, _ = x.size()
        hs_forward = torch.zeros(x.size(0), self.hidden_size).to(device)
        cs_forward = torch.zeros(x.size(0), self.hidden_size).to(device)
        hs_backward = torch.zeros(x.size(0), self.hidden_size).to(device)
        cs_backward = torch.zeros(x.size(0), self.hidden_size).to(device)
        
        forward = []
        backward = []
        
        # Forward
        for t in range(seq_sz):
            hs_forward, cs_forward = self.plstm_cell_forward(t, x, ts, (hs_forward, cs_forward))
            forward.append(hs_forward.unsqueeze(0))
        # Backward
        for t in reversed(range(seq_sz)):
            hs_backward, cs_backward = self.plstm_cell_backward(t, x, ts, (hs_backward, cs_backward))
            backward.append(hs_backward.unsqueeze(0))
        
        #doesn't matter actually
        h_t, c_t = torch.cat((hs_forward, hs_backward), 1), torch.cat((cs_forward, cs_forward), 1)
        
        hidden_seq = []
        for fwd, bwd in zip(forward, backward):
            hidden_seq.append(torch.cat((fwd, bwd), 2))
        hidden_seq = torch.cat(hidden_seq, dim=0)
        # reshape from shape (sequence, batch, feature) to (batch, sequence, feature)
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()
        return hidden_seq, (h_t, c_t)

In [3]:
class Net(nn.Module):
    def __init__(self, weights):
        super(Net, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(weights, freeze=False)
        #self.lstm = PLSTM(100, 128).to(device)
        self.blstm = BiPLSTM(100, 128).to(device)
        self.drop_out1 = nn.Dropout()
        self.maxpool = nn.MaxPool1d(3, stride=2)
        self.hidden1 = nn.Linear(127*2000, 128)
        self.hidden2 = nn.Linear(128,32)
        self.hidden3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        ts = x.to(device)
        x = self.embedding(x).float()
        x, (hn, cn) = self.blstm(x, ts)
        x = self.drop_out1(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.hidden1(x))
        x = x.view(x.size(0), -1)
        x = self.hidden2(x)
        x = x.view(x.size(0), -1)
        x = self.hidden3(x)
        return x

In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    return acc

#torch.cuda.set_per_process_memory_fraction(0.5, 0)
print(device)
    
    
net = Net(weights).to(device)
# print(weights.shape)

criterion = nn.BCEWithLogitsLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.001)
epochs = 60

def train(model, device, train_loader, optimizer, epoch, epoch_loss, epoch_acc):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target.unsqueeze(1))
        acc = binary_acc(output, target.unsqueeze(1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        print(f'Epoch {epoch+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

for epoch in range(epochs):
    epoch_loss = 0
    epoch_acc = 0
    train(net, device, train_loader, optimizer, epoch, epoch_loss, epoch_acc)

PATH = 'trained_model/VLC_PLSTM_'
torch.save(net.state_dict(), PATH + 'model.h5')


In [5]:
#obtain representations
def ObtainRepresentations_by_batch_size(input_sequences, feature_model, BATCH_SIZE):
    num_batches_per_epoch = int((len(input_sequences) - 1) / BATCH_SIZE) + 1
    data_size = len(input_sequences)
    representations_total = []
    model = feature_model
    for batch_num in range(num_batches_per_epoch):
        start_index = batch_num * BATCH_SIZE
        end_index = min((batch_num + 1) * BATCH_SIZE, data_size)
        print ("-------start_index------------")
        print (start_index)
        print ("-------end_index------------")
        print (end_index)
        print(input_sequences[start_index: end_index].shape)
        representations = model(input_sequences[start_index: end_index])
        representations_total = representations_total + representations.tolist()
    return np.asarray(representations_total)

net = Net(weights).to(device)
PATH = 'trained_model/VLC_PLSTM_'
model_path = PATH + 'model.h5'
net.load_state_dict(torch.load(model_path))
net.eval()

class Feature_Net(nn.Module):
    def __init__(self, net, weights):
        super(Feature_Net, self).__init__()
        self.embedding = net.embedding
        self.blstm = net.blstm
        self.drop_out1 = net.drop_out1
        self.maxpool = net.maxpool
        self.hidden1 = net.hidden1
        self.relu = net.relu
    def forward(self, x):
        ts = x.to(device)
        x = self.embedding(x).float()
        x, (hn, cn) = self.blstm(x, ts)
        x = self.drop_out1(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.hidden1(x))
        return x

feature_model = Feature_Net(net, weights).to(device)
feature_model.eval()

tokenizer = LoadToknizer(tokenizer_saved_path + 'tokenizer.pickle')
data_path_repre = 'data/ALL/VLC/Vul/'
data_list, data_list_id = loadData(data_path_repre) 
data_sequence = tokenizer.texts_to_sequences(data_list)
data_list_pad = padding(data_sequence)
data_list_pad = torch.Tensor(data_list_pad).long().to(device)
print(data_list_pad.shape)
obtained_repre = ObtainRepresentations_by_batch_size(data_list_pad, feature_model, 128)
print("Saving the obtained representations....") 
if not os.path.exists(repre_saved_path): os.makedirs(repre_saved_path)
SavedPickle(repre_saved_path + "VLC_Vul.pkl", obtained_repre)
# When the obtained representations are 2-D arrays, we can also save them in a CSV file.
#ListToCSV(obtained_repre, self.paras.saved_path)
print("The obtained representations are saved in: " + str(repre_saved_path) + ".")

[INFO] Loading data from /home/swj/VD/data/ALL/VLC/Vul/....
[INFO] The length of the loaded data list is : 44
torch.Size([44, 2000])
-------start_index------------
0
-------end_index------------
44
torch.Size([44, 2000])
Saving the obtained representations....
The obtained representations are saved in: Representation/.


In [ ]:
#eval the model with only PLSTM used
net = Net(weights).to(device)
PATH = 'trained_model/FFmpeg_PLSTM_'
model_path = PATH + 'model.h5'
net.load_state_dict(torch.load(model_path))

testloader = test_set
# since we're not training, we don't need to calculate the gradients for our outputs
predicted_classes = []
test_set_y = testloader[1]
test_accuracy = 0

y_pred_list = []
model = net
model.eval()
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(test_loader):
        data, target = data.to(device), target.to(device)
        y_test_pred = model(data)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

confusion_matrix(y_test, y_pred_list)
print(classification_report(y_test, y_pred_list))
